In [7]:
import os
import re
import pywintypes
import win32file
import win32con
import datetime
import piexif
from PIL import Image
from PIL.ExifTags import TAGS

# ------------------ CONFIG ------------------
ROOT_DIR = r'c:\Users\kuste\Desktop\project\Arhiv slik'
IMAGE_EXTS = ['.jpg', '.jpeg', '.JPG', '.JPEG']
VIDEO_EXTS = ['.mp4', '.MP4']
YEAR_PATTERN = re.compile(r'\b(20[0-2][0-9]|2030)\b')  # 2000-2030
FALLBACK_MONTH = 5  # May
FALLBACK_DAY = 1    # 1st
# --------------------------------------------

def get_folder_year(folder_name):
    match = YEAR_PATTERN.search(folder_name)
    return int(match.group()) if match else None

def get_image_date(image_path):
    try:
        with Image.open(image_path) as img:
            exif = img._getexif()
            if exif:
                for tag, value in exif.items():
                    if TAGS.get(tag) == 'DateTimeOriginal':
                        return datetime.datetime.strptime(value, '%Y:%m:%d %H:%M:%S')
    except Exception as e:
        print(f"⚠️ Error reading {image_path}: {str(e)}")
    return None

def get_video_date(video_path):
    try:
        creation_time = os.path.getctime(video_path)
        return datetime.datetime.fromtimestamp(creation_time)
    except Exception as e:
        print(f"⚠️ Error reading {video_path}: {str(e)}")
    return None

def set_filesystem_dates(file_path, new_date):
    try:
        win_time = pywintypes.Time(new_date)
        handle = win32file.CreateFile(
            file_path,
            win32con.GENERIC_WRITE,
            win32con.FILE_SHARE_READ | win32con.FILE_SHARE_WRITE,
            None,
            win32con.OPEN_EXISTING,
            win32con.FILE_ATTRIBUTE_NORMAL,
            None
        )
        win32file.SetFileTime(handle, win_time, win_time, win_time)
        handle.Close()
        return True
    except Exception as e:
        print(f"❌ Failed to update filesystem dates for {file_path}: {e}")
    return False

def update_image_metadata(image_path, new_date):
    try:
        exif_dict = piexif.load(image_path)
        date_str = new_date.strftime("%Y:%m:%d %H:%M:%S")
        exif_dict['Exif'][piexif.ExifIFD.DateTimeOriginal] = date_str.encode('utf-8')
        exif_bytes = piexif.dump(exif_dict)
        piexif.insert(exif_bytes, image_path)
        return True
    except Exception as e:
        print(f"❌ Failed to update EXIF for {image_path}: {e}")
    return False

def is_image(file_path):
    return file_path.lower().endswith(tuple(IMAGE_EXTS))

def is_video(file_path):
    return file_path.lower().endswith(tuple(VIDEO_EXTS))

def process_file(file_path, force_date, folder_year):
    if force_date:
        if is_image(file_path):
            update_image_metadata(file_path, force_date)
        return set_filesystem_dates(file_path, force_date)

    if is_image(file_path):
        img_date = get_image_date(file_path)
        if img_date:
            update_image_metadata(file_path, img_date)
            return set_filesystem_dates(file_path, img_date)
    elif is_video(file_path):
        vid_date = get_video_date(file_path)
        if vid_date:
            return set_filesystem_dates(file_path, vid_date)

    fallback_date = datetime.datetime(folder_year, FALLBACK_MONTH, FALLBACK_DAY) if folder_year else None
    if fallback_date:
        if is_image(file_path):
            update_image_metadata(file_path, fallback_date)
        return set_filesystem_dates(file_path, fallback_date)

    return False

def process_folder(folder_path):
    folder_name = os.path.basename(folder_path)
    folder_year = get_folder_year(folder_name)
    updated_count = 0
    force_date = None

    # Check for date.txt
    date_file = os.path.join(folder_path, 'date.txt')
    if os.path.exists(date_file):
        try:
            with open(date_file, 'r', encoding='utf-8') as f:
                date_str = f.readline().strip()
                date_str = ''.join(c for c in date_str if c.isprintable())
                force_date = datetime.datetime.strptime(date_str, "%A, %d %B %Y")
                print(f"📅 Forced date from date.txt: {force_date}")
        except Exception as e:
            print(f"⚠️ Could not read date.txt: {e}")

    for filename in sorted(os.listdir(folder_path)):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and (is_image(file_path) or is_video(file_path)):
            if process_file(file_path, force_date, folder_year):
                print(f"✔️ {filename} updated")
                updated_count += 1

    return updated_count

def main():
    total_updated = 0
    for root, dirs, files in os.walk(ROOT_DIR):
        print(f"\n📁 Processing: {root}")
        updated = process_folder(root)
        total_updated += updated
        print(f"✅ {updated} files updated in {root}")

    print(f"\n🎉 All done! Total files updated: {total_updated}")

if __name__ == "__main__":
    main()



📁 Processing: c:\Users\kuste\Desktop\project\Arhiv slik
✅ 0 files updated in c:\Users\kuste\Desktop\project\Arhiv slik

📁 Processing: c:\Users\kuste\Desktop\project\Arhiv slik\!Prema gore kranjske
✔️ IMG_20210920_111621.jpg updated
✔️ IMG_20210920_112058.jpg updated
✔️ IMG_20210921_172137.jpg updated
✔️ IMG_20210921_173506.jpg updated
✔️ IMG_20210921_175826.jpg updated
✔️ IMG_20210923_130216.jpg updated
✔️ IMG_20210923_204824-je uredil(-a)_.jpg updated
✔️ IMG_20210923_204824.jpg updated
✔️ IMG_20210923_204837-je uredil(-a)_.jpg updated
✔️ IMG_20210923_204837.jpg updated
✔️ VID_20210922_180333~2.mp4 updated
✔️ received_3173426426226982.jpeg updated
✅ 12 files updated in c:\Users\kuste\Desktop\project\Arhiv slik\!Prema gore kranjske

📁 Processing: c:\Users\kuste\Desktop\project\Arhiv slik\Arhiv
✔️ .trashed-1739827208-WallX_1349137_1080x1920.jpeg updated
✔️ 03f53d3f919db3b017d5b66e19dece65.jpg updated
✔️ 1119991.jpg updated
✔️ 137374_original_2738x3000.jpg updated
✔️ 1d14f37c8047758a4610

c:\Users\kuste\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:3402: DecompressionBombWarning: Image size (108576768 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


✔️ IMG20241110111515.jpg updated
❌ Failed to update EXIF for c:\Users\kuste\Desktop\project\Arhiv slik\Photos from 2024-25\IMG20241110111524.jpg: Given data isn't JPEG.
✔️ IMG20241110111524.jpg updated
❌ Failed to update EXIF for c:\Users\kuste\Desktop\project\Arhiv slik\Photos from 2024-25\IMG20241110111533.jpg: Given data isn't JPEG.
✔️ IMG20241110111533.jpg updated
❌ Failed to update EXIF for c:\Users\kuste\Desktop\project\Arhiv slik\Photos from 2024-25\IMG20241110123438.jpg: Given data isn't JPEG.
✔️ IMG20241110123438.jpg updated
✔️ IMG20241110123857.jpg updated
✔️ IMG20241110123921.jpg updated
❌ Failed to update EXIF for c:\Users\kuste\Desktop\project\Arhiv slik\Photos from 2024-25\IMG20241110123947.jpg: Given data isn't JPEG.
✔️ IMG20241110123947.jpg updated
❌ Failed to update EXIF for c:\Users\kuste\Desktop\project\Arhiv slik\Photos from 2024-25\IMG20241110123959.jpg: Given data isn't JPEG.
✔️ IMG20241110123959.jpg updated
❌ Failed to update EXIF for c:\Users\kuste\Desktop\proje